In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
from math import *
import fmlib
from grading_tools import *

# Exercises

## Exercise

Write a function to price a call option in the jump diffusion model. You should include
sufficient terms in the power series to ensure that your answer is accurate to roughly one part
in a million.

In [2]:
def jump_diffusion_call_price( S, K, T, r, sigma, lbda, j):
    ### BEGIN SOLUTION
    S = np.array(S)
    term = np.ones(S.shape)
    total = np.zeros(S.shape)
    n = 0
    mu_tilde = r + lbda*(1-j)
    while np.any(abs(term)>1e-7*abs(total)):
        V = fmlib.black_scholes_call_price(S,0,
j**(-n)*K,T,mu_tilde, sigma)
        term = ((j*lbda*T)**n)/factorial(n) * exp( -j*lbda*T) * V
        total = total + term
        n = n+1
    return total
    ### END SOLUTION

In [3]:
# Choose some model parameters for our tests
r = 0.02
S0 = 150
sigma = 0.2
T = 1
lbda = 1
j = 0.9

# The stock is the same thing as a call with a strike of 0, so for very
# low strikes the price should be nearly S0
actual = jump_diffusion_call_price( S0, 0.000001, T, r, sigma, lbda, j )
np.testing.assert_almost_equal(S0,actual,decimal=2)
auto_marking_message()

Auto marking message: 🏆 Correct


In [4]:
# If the jump size is 1, we should get back the Black-Scholes price
actual = jump_diffusion_call_price( S0, S0, T, r, sigma, lbda, 1 )
expected = fmlib.black_scholes_call_price( S0, 0, S0, T, r, sigma )
np.testing.assert_almost_equal(actual,expected,decimal=2)
auto_marking_message()

Auto marking message: ✔️ Correct


## Exercise

To simulate the jump diffusion model on a grid of size $\delta t$, let $Z_t=\log(S_t)$. We may simulate $Z_t$ as
$$
Z_{t+\delta t}=Z_t + (\tilde{\mu}-\tfrac{1}{2}\sigma^2)\, \delta t + \sigma (\delta t)^{\frac{1}{2}} \epsilon_t + n_{t,t+\delta t} \log(j)
$$
where the $\epsilon_t$ are independent standard normal random variables and the
$n_{t,t+\delta t}$ are independent Poisson random variables with intensity parameter $\lambda \, \delta t$.
The variables $n_{t,t+\delta t}$ represent the number of jumps between time $t$ and time $t+\delta t$.

In [5]:
def simulate_jump_diffusion( S0, T, mu_twiddle, sigma, lbda, j, n_steps, n_paths ):
    t = np.linspace(0,T,n_steps+1)
    ### BEGIN SOLUTION
    dt = T/n_steps
    Z = np.zeros((n_paths,n_steps+1))
    Z[:,0]=np.log(S0)
    for i in range(0, n_steps):
        epsilon = np.random.normal(size=(n_paths))
        jumps = np.random.poisson(lbda*dt, n_paths )
        Z[:,i+1]=Z[:,i] + (mu_twiddle-0.5*sigma**2)*dt + sigma*np.sqrt(dt)*epsilon + np.log(j)*jumps
    S = np.exp(Z)
    ### END SOLUTION
    return S, t

In [6]:
# The discounted expected stock price in the Q-model at time T
# should equal S0 by the consistency condition
# We check the answer is within a 99.9% confidence interval
np.random.seed(0)
n_steps = 365
n_paths = 100000
mu_twiddle = r + lbda*(1-j)
S,t = simulate_jump_diffusion(S0,T,mu_twiddle,sigma, lbda, j, n_steps, n_paths)

discounted_expectation = np.exp(-r*T)*np.mean(S[:,-1])
sd = np.exp(-r*T)*np.std(S[:,-1])/np.sqrt(n_paths)
alpha = scipy.stats.norm.ppf(0.0005)
assert discounted_expectation>S0+alpha*sd
assert discounted_expectation<S0-alpha*sd
auto_marking_message()

Auto marking message: 😍 Correct


In [7]:
# In fact the simulation method is exact, so only one step is needed
S,t = simulate_jump_diffusion(S0,T,mu_twiddle,sigma, lbda, j, 1, n_paths)
discounted_expectation = np.exp(-r*T)*np.mean(S[:,-1])
assert discounted_expectation>S0+alpha*sd
assert discounted_expectation<S0-alpha*sd
auto_marking_message()

Auto marking message: ✨ Correct


## Exercise

Write a general Monte Carlo pricing routine for the jump diffusion model which works
with arbitrary payoff functions. Your code should return a 99% confidence interval for the price
exactly as the function `price_by_monte_carlo` in notebook 7 of the previous topic
did for the Black-Scholes model.

In [8]:
def price_by_monte_carlo_jd( S0, r, sigma, lbda, j, T, n_steps,n_paths, payoff_function):
    ### BEGIN SOLUTION
    mu_twiddle = r + lbda*(1-j)
    S,t = simulate_jump_diffusion( S0, T, mu_twiddle, sigma, lbda, j, n_steps, n_paths )
    payoffs = payoff_function(S)
    p = 99
    alpha = scipy.stats.norm.ppf((1-p/100)/2)
    price = np.exp(-r*T)*np.mean( payoffs )
    sigma_sample = np.exp(-r*T) * np.std( payoffs )
    lower = price + alpha*sigma_sample/np.sqrt(n_paths)
    upper = price - alpha*sigma_sample/np.sqrt(n_paths)
    ### END SOLUTION
    return lower, upper

In [9]:
def price_call_by_monte_carlo_jd( S0, K, T, r, sigma, lbda, j, n_steps,n_paths ):
    # Define the payoff function, it takes an array 
    def call_payoff( S ):
        S_T = S[:,-1]
        return np.maximum( S_T-K, 0 )
    return price_by_monte_carlo_jd(S0, r, sigma, lbda, j, T, n_steps, n_paths, call_payoff )

def test_price_call_by_monte_carlo_jd():
    np.random.seed(0)
    # Only one step is needed to price a call option
    n_steps = 1
    K = S0
    low,high = price_call_by_monte_carlo_jd(S0, K, T, r,sigma,lbda,j, n_steps, n_paths)
    expected = jump_diffusion_call_price(S0,K,T,r,sigma, lbda, j)
    assert low<expected
    assert expected<high

test_price_call_by_monte_carlo_jd()
auto_marking_message()

Auto marking message: 🌟 Correct


## Exercise

Use a Monte Carlo method with 100000 steps to estimate the price of an Asian call option with strike $140$ and maturity $1$ in a jump diffusion model with parameters $S_0$, $r$, $\sigma$, $\lambda$ and $j$
as defined in the cells above. When computing the payoff of the Asian option, you should compute the average price at the end of each day. Store your estimated value in a variable called `asian_price`.

In [10]:
K = 140
T = 1

def asian_call_payoff( S, K ):
    S_bar = np.mean(S,axis=1)
    return np.maximum( S_bar-K, 0 )

def price_asian_call_by_monte_carlo_jd( S0, r, sigma, lbda, j, K, T, n_steps, n_paths ):
    def payoff_fn(S):
        return asian_call_payoff(S,K)
    return price_by_monte_carlo_jd(S0, r, sigma, lbda,j, T, n_steps, n_paths, payoff_fn )

n_paths = 100000
low,high = price_asian_call_by_monte_carlo_jd(S0,r,sigma,lbda, j, K, T, T*365, n_paths)
asian_price = 0.5*(low+high)
print('{}-{}'.format(low,high)) 

14.257029512523063-14.509711083970336


In [11]:
assert asian_price>0
### BEGIN HIDDEN TESTS
assert asian_price>14.1
assert asian_price<14.7
### END HIDDEN TESTS
auto_marking_message()

Auto marking message: 🌟 Correct
